In [80]:
import os
import re
from glob import glob

import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table, QTable

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

# First parse/load the data 

##  ELVIS

In [87]:
def read_elvis_z0(fn):
    tab = QTable.read(os.path.join(elvis_data_dir, fn), format='ascii.commented_header',data_start=0, header_start=1)
    
    col_name_re = re.compile(r'(.*?)(\(.*\))?$')
    for col in tab.columns.values():
        match = col_name_re.match(col.name)
        if match:
            nm, unit = match.groups()
            if nm != col.name:
                col.name = nm
            if unit is not None:
               col.unit = u.Unit(unit[1:-1]) # 1:-1 to get rid of the parenthesis
        
    return tab

def load_elvii(data_dir=os.path.abspath('elvis_data/'), isolated=False):
    tables = {}
    
    fntoload = glob(os.path.join(data_dir, '*.txt'))
    for fn in fntoload:
        simname = os.path.split(fn)[-1][:-4]
        if simname.startswith('i'):
            if not isolated:
                continue
        else:
            if isolated:
                continue
        print('Loading', fn)
        tables[simname] = read_elvis_z0(fn)
    return tables

In [88]:
elvii_pairs = load_elvii(isolated=False)

Loading /Users/erik/projects/m31-galfa/elvis/data/Burr&Hamilton.txt
Loading /Users/erik/projects/m31-galfa/elvis/data/Charybdis&Scylla.txt
Loading /Users/erik/projects/m31-galfa/elvis/data/Cher&Sonny.txt
Loading /Users/erik/projects/m31-galfa/elvis/data/Hall&Oates.txt
Loading /Users/erik/projects/m31-galfa/elvis/data/Kauket&Kek.txt
Loading /Users/erik/projects/m31-galfa/elvis/data/Lincoln&Douglas.txt
Loading /Users/erik/projects/m31-galfa/elvis/data/Romulus&Remus.txt
Loading /Users/erik/projects/m31-galfa/elvis/data/Siegfried&Roy.txt
Loading /Users/erik/projects/m31-galfa/elvis/data/Taurus&Orion.txt
Loading /Users/erik/projects/m31-galfa/elvis/data/Thelma&Louise.txt
Loading /Users/erik/projects/m31-galfa/elvis/data/Venus&Serana.txt
Loading /Users/erik/projects/m31-galfa/elvis/data/Zeus&Hera.txt


## GALFA Sensitivity Maps 